In [ ]:
from cyschoolhousesuite import *
from selenium.webdriver.common.action_chains import ActionChains
import numpy as np
import pdb

In [ ]:
stdnt_sctn_df = get_report("00O1a000002rl1s")

stdnt_sctn_df['Program: Program Name'].loc[stdnt_sctn_df['Section Name'].str.contains("Grade") & stdnt_sctn_df['Section Name'].str.contains("Math")] = "Tutoring: Math"
stdnt_sctn_df['Program: Program Name'].loc[stdnt_sctn_df['Section Name'].str.contains("Grade") & stdnt_sctn_df['Section Name'].str.contains("Literacy")] = "Tutoring: Literacy"

stdnt_sctn_df['Key'] = stdnt_sctn_df['Student: Student Id'] + stdnt_sctn_df['Program: Program Name']

gb_df = stdnt_sctn_df.loc[stdnt_sctn_df['Section Name'].str.contains("Grade")]
fl_df = stdnt_sctn_df.loc[stdnt_sctn_df['Section Name'].str.contains("Tutoring")]

# Create group object
sums = fl_df[['Key', 
              'In School/ Extended Learning',
              'Student: Student Id',
              'Active']].groupby(['Key',
                                  'In School/ Extended Learning',
                                  'Student: Student Id']).sum()
# Apply sum function
sums = sums.reset_index()
enrolled_in_fl = sums.loc[(sums['In School/ Extended Learning']=='In School') & (sums['Active']>0)]
unenrolled_from_all_fl = sums.loc[(sums['In School/ Extended Learning']=='In School') & (sums['Active']==0)]

In [ ]:
add_to_gb = enrolled_in_fl.loc[~enrolled_in_fl['Key'].isin(gb_df['Key'])]
rmv_frm_gb = gb_df.loc[gb_df['Key'].isin(unenrolled_from_all_fl['Key'])]

df = add_to_gb.merge(fl_df[['Key', 'Account Name', 'Student: Grade', 'Program: Program Name']].drop_duplicates(subset='Key'), how='left', on='Key')

df['Program: Program Name'].loc[df['Program: Program Name'].str.contains("Literacy")] = "ELA/Literacy"
df['Program: Program Name'].loc[df['Program: Program Name'].str.contains("Math")] = "Math"

df['New Section'] = df['Account Name'] + " - Grade " + df['Student: Grade'].astype(str) + " - " + df['Program: Program Name']

print("{} students to add to gradebook sections".format(len(add_to_gb)))

# CY-351885 Dulles School of Excellence - Grade 4 - ELA/Literacy ... start date entered as 12/05/2017
# CY-351892 Dulles School of Excellence - Grade 8 - Math ... start date entered as 12/05/2017
# CY-351651 too
# CY-352742 too
# Run audit for students with section start dates greater than 7/1/2017

In [ ]:
# Add Students to Gradebook Sections
driver = get_driver()
open_cyschoolhouse18(driver)

wait = 10
entry_date = "07-01-2017"

for school in df['Account Name'].unique():
    # iterate by school
    print("Starting: " + school)
    driver.get("https://c.na24.visual.force.com/apex/core_ChangeSchool")
    driver.find_element_by_xpath("//select[@name='j_id0:j_id30:j_id32']/option[text()=\"%s\"]" % school).click()
    sleep(2)
    driver.find_element_by_xpath("//input[contains(@class, 'btn') and contains(@value, 'Change')]").click()
    sleep(2)
    
    for index, row in df.loc[df['Account Name']==school].iterrows():
        # iterate by student in school
        print('Starting: student: {}'.format(row['New Section']))
        driver.get("https://c.na24.visual.force.com/apex/Scheduler_SectionWizard")

        driver.find_element_by_xpath("//*[@id='thePage:j_id70:j_id71:j_id74']").click()
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//*[@id='thePage:j_id70:j_id71:j_id123']")))
        driver.find_element_by_xpath("//*[@id='thePage:j_id70:j_id71:j_id123']").click()
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, ".//input[contains(@aria-controls, 'DataTables_Table_0')]")))
        # Search for and select student
        try:
            driver.find_element_by_xpath(".//input[contains(@aria-controls, 'DataTables_Table_0')]").clear()
            driver.find_element_by_xpath(".//input[contains(@aria-controls, 'DataTables_Table_0')]").send_keys(row['Student: Student Id'])
            driver.find_element_by_xpath(".//input[contains(@class, 'searchResultCb')]").click()
            driver.find_element_by_xpath("//*[@id='thePage:j_id70:j_id71:j_id144']").click()
        except:
            print('Failed: Could not select student: ' + row['Student: Student Id'])
            continue
        # Enter section details
        try:
            WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.XPATH, "//input[contains(@aria-controls, 'thePage:j_id70:j_id71:j_id182')]")))
            # ensure section name is in the 'Available Sections' table rather than 'Current Sections'
            driver.find_element_by_xpath("//*[contains(@class, 'tablesorter dataTable') and contains(@title, 'Available Sections')][.//*[contains(.,\"%s\")]]"% row['New Section'])
            driver.find_element_by_xpath("//input[contains(@aria-controls, 'thePage:j_id70:j_id71:j_id182')]").send_keys(row['New Section'][-4:])
            driver.find_element_by_xpath("//span[contains(text(), \"%s\") and contains(@id, 'thePage')]" % row['New Section']).click()
            ActionChains(driver).key_down(Keys.SHIFT).send_keys(Keys.TAB).send_keys(Keys.TAB).key_up(Keys.SHIFT).send_keys(Keys.SPACE).perform()
            ActionChains(driver).send_keys(Keys.TAB).send_keys(Keys.TAB).send_keys(entry_date).send_keys(Keys.RETURN).perform()
            # press save
            driver.find_element_by_xpath("//input[contains(@type, 'button') and contains(@value, 'SAVE')]").send_keys(Keys.SPACE)
        except:
            print('Failed: Could not select section: ' + row['New Section'])
            continue

        fancy_box_wait(driver, waittime=10)

        # Confirm successful
        while 'Changes saved!' not in driver.find_element_by_tag_name("body").text:
            sleep(.5)

driver.quit()